In [1]:
print("Ok")

Ok


In [2]:
!pip install torch==1.2.0 torchvision==0.4.0 -f


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-f option requires 1 argument


In [3]:
%pwd

'/Users/preetyrai/End_To_End_Medical_RAG_Chatbot/research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'/Users/preetyrai/End_To_End_Medical_RAG_Chatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [8]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 4939


In [12]:
# text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
!pip install -U sentence-transformers

In [15]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [16]:
embeddings = download_hugging_face_embeddings()

/var/folders/kx/d1gxzt7s111d0z9bjypgv2980000gn/T/ipykernel_76544/2477585721.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/opt/anaconda3/envs/medibot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))



Length 384


In [18]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

download_hugging_face_embeddings()

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
# embeddings = download_hugging_face_embeddings()

In [21]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [22]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [25]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "internationlawbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "internationlawbot",
    "metric": "cosine",
    "host": "internationlawbot-jjxj3mh.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [26]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [27]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [28]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [29]:
docsearch

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [31]:
retrieved_docs = retriever.invoke("What is US Law?")


In [32]:
retrieved_docs

[Document(id='725befff-0fc3-4079-b953-a298ef03e3c4', metadata={'creationdate': '2017-05-04T12:01:01-04:00', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'moddate': '2017-05-05T14:58:51-04:00', 'page': 3.0, 'page_label': 'iv', 'producer': 'Adobe PDF Library 9.9', 'source': 'Data/international_law.pdf', 'total_pages': 681.0, 'trapped': '/False'}, page_content='Copyright © United Nations, 2017\nThe fundamental goal of the International Law Handbook is to promote the teaching and dissemina-\ntion of international law around the world under the United Nations Programme of Assistance in \nthe Teaching, Study, Dissemination and Wider Appreciation of International Law. This publication \nmay be reproduced for academic purposes to further this goal anywhere in the world. It may be'),
 Document(id='6c160af9-c2bd-4067-92d8-a3722e6ce103', metadata={'creationdate': '2017-05-04T12:01:01-04:00', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'moddate': '2017-05-05T14:58:51-04:00', 'page': 116.0, 'page_label': 

In [40]:
groq_api_key= "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"

In [45]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500)

from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model_name="deepseek-r1-distill-llama-70b")


In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [47]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [51]:
# from groq import Groq
# client = Groq()
# completion = client.chat.completions.create(
#     model="deepseek-r1-distill-llama-70b",
#     messages=[
#         {
#             "role": "user",
#             "content": "Explain why fast inference is critical for reasoning models"
#         }
#     ]
# )
# print(completion.choices[0].message.content)


In [49]:
response = rag_chain.invoke({"input": "what is us law?"})
print(response["answer"])


<think>
Okay, the user is asking, "What is US law?" I need to answer based on the provided context. Let me look through the context again. The context is about international law, specifically from a UN publication. It discusses subjects of international law, states parties, and committees. There's nothing here about US law specifically. The context is focused on international legal frameworks, not domestic laws of individual countries. Since the question is about US law, which isn't covered in the context, I can't provide an answer from the given information. I should respond that I don't know.
</think>

I don't know.


In [ ]:
response = rag_chain.invoke({"input": "United Nations Convention on jurisdictional immunities of States and their property"})
print(response["answer"])


<think>
Okay, I need to answer the question about the United Nations Convention on jurisdictional immunities of States and their property. Let me read the context carefully.

The context mentions that the Convention was done in New York on December 2, 2004, and it's not yet in force. It was adopted by General Assembly resolution 59/38. The key principle is that States have immunity from the jurisdiction of another State's courts, with exceptions as per the Convention.

So the main points are: when it was done, where, that it's not in force yet, and the main principle about state immunity.
</think>

The United Nations Convention on Jurisdictional Immunities of States and Their Property was adopted on December 2, 2004, in New York, under General Assembly resolution 59/38. It is not yet in force. The Convention establishes that States and their property are immune from the jurisdiction of another State's courts, with specific exceptions outlined within the Convention.
